<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-GSC-Functions" data-toc-modified-id="Import-GSC-Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import GSC Functions</a></span><ul class="toc-item"><li><span><a href="#Check-sites-URL" data-toc-modified-id="Check-sites-URL-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Check sites URL</a></span></li></ul></li><li><span><a href="#Create-original-table-for-historical-data-ETL" data-toc-modified-id="Create-original-table-for-historical-data-ETL-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create original table for historical data ETL</a></span><ul class="toc-item"><li><span><a href="#Time-variables" data-toc-modified-id="Time-variables-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Time variables</a></span></li><li><span><a href="#Get-historical-data" data-toc-modified-id="Get-historical-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get historical data</a></span></li><li><span><a href="#Classificate-brand,-generic-and-cameras" data-toc-modified-id="Classificate-brand,-generic-and-cameras-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Classificate brand, generic and cameras</a></span></li><li><span><a href="#Parse-dates" data-toc-modified-id="Parse-dates-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Parse dates</a></span></li><li><span><a href="#Save-csv" data-toc-modified-id="Save-csv-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Save csv</a></span></li></ul></li></ul></div>

In [1]:
import calendar
import datetime
import json
import pandas as pd
import re
import requests

from google.oauth2 import service_account
from googleapiclient.discovery import build

pd.set_option('max_colwidth', 150)

# Import GSC Functions

In [2]:
%run 'GSC-Functions.ipynb'

## Check sites URL

In [3]:
#------------------------------------------- get verified sites ------------------------------------------

verified_sites_urls

['https://www.verisure.com.ar/',
 'https://www.verisure.fi/',
 'https://www.verisure.fr/',
 'https://www.securitasdirect.pt/',
 'https://www.verisure.be/',
 'https://www.verisure.co.uk/',
 'https://www.verisure.no/',
 'https://www.verisure.it/',
 'https://www.verisure.pe/',
 'https://www.verisure.cl/',
 'https://www.verisure.de/',
 'https://www.verisure.dk/',
 'https://www.verisure.se/',
 'https://www.verisure.com.br/',
 'https://www.verisure.nl/',
 'https://www.securitasdirect.es/']

# Create original table for historical data ETL

## Time variables

In [4]:
today, first_available, endDate, startDate

(datetime.datetime(2022, 4, 25, 10, 0, 38, 390222),
 datetime.datetime(2020, 12, 31, 10, 0, 38, 390222),
 '2022-04-25',
 '2021-01-01')

## Get historical data

In [5]:
#------------------------------------------- get chunk 1 historical data - from 01-01-2021 -------------------------------

service = connect(key)
maxRows = 25000
i = 0
payload = {
    'startDate': '2021-01-01',
    'endDate': '2021-04-30',
    'dimensions': ['date', 'query', 'country', 'device'],
    'rowLimit': maxRows,
    'startRow': 0}

results = []

for s in verified_sites_urls:
    print(s)
    df_s = paginated(service, s, payload)
    payload['startRow'] = 0
    results.append(df_s)
    print('\n')

df_hist1 = pd.concat(results)
df_hist1

https://www.verisure.com.ar/
Working on loop 0
Working on loop 1
Working on loop 2
Working on loop 3
Working on loop 4
Working on loop 5
Working on loop 6
Working on loop 7
Working on loop 8
Working on loop 9


https://www.verisure.fi/
Working on loop 0
Working on loop 1
Working on loop 2
Working on loop 3
Working on loop 4
Working on loop 5


https://www.verisure.fr/
Working on loop 0
Working on loop 1
Working on loop 2
Working on loop 3
Working on loop 4
Working on loop 5
Working on loop 6
Working on loop 7
Working on loop 8
Working on loop 9
Working on loop 10
Working on loop 11
Working on loop 12
Working on loop 13
Working on loop 14
Working on loop 15
Working on loop 16
Working on loop 17
Working on loop 18
Working on loop 19
Working on loop 20
Working on loop 21
Working on loop 22
Working on loop 23
Working on loop 24
Working on loop 25
Working on loop 26
Working on loop 27
Working on loop 28
Working on loop 29
Working on loop 30
Working on loop 31
Working on loop 32
Working on l

,date,query,country,device,clicks,impressions,ctr,position,site_country
0,2021-03-02,verisure argentina,arg,MOBILE,32,110,29.09,1.95,ar/
1,2021-03-05,verisure,arg,DESKTOP,32,52,61.54,1.52,ar/
2,2021-03-09,verisure,arg,DESKTOP,30,76,39.47,1.30,ar/
3,2021-02-24,verisure,arg,DESKTOP,25,82,30.49,1.62,ar/
4,2021-03-08,verisure,arg,DESKTOP,25,74,33.78,1.70,ar/
...,...,...,...,...,...,...,...,...,...
1403338,2021-04-30,zerovision verisure,chl,MOBILE,0,1,0.00,3.00,es/
1403339,2021-04-30,zerovision verisure,prt,TABLET,0,1,0.00,10.00,es/
1403340,2021-04-30,zonas mas peligrosas de españa,esp,DESKTOP,0,1,0.00,7.00,es/
1403341,2021-04-30,¿por que me siento sola? en tenerife,bra,DESKTOP,0,1,0.00,143.00,es/


## Classificate brand, generic and cameras

In [6]:
df_hist1 = query_cats(df_hist1)

## Parse dates

In [7]:
df_hist1 = date_datetime(df_hist1)

## Save csv

In [8]:
#------------------------------------------- save chunk to local file ----------- -------------------------------

df_hist1.to_csv('files/GSC-01012021-to-04192025-Chunk1.csv', index=False)